In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "_paper_2"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol

using StaticArrays
using Rotations
using Distances
using LinearAlgebra
using CairoMakie
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [ ]:
# base = "../../Data/hpc_out/rwm_ma_3_6r7m/"
# for dir in readdir(base)
#     for file in readdir("$(base)$(dir)")
#         if split(file, ".")[end] == "jld2"
#             @load "$(base)$(dir)/$(file)" input output
#             E_min = minimum(output["Es"])
#             # if E_min < 12200.0
#             #     println("$(E_min) | $(base)$(dir)/$(file)")
#             # end
#         end
#     end
# end

folder = "../../Data/hpc_out/rwm_ma_3_6r7m/"
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        @load "$(folder)$(file)" input output
        E_min = minimum(output["Es"])
        if E_min < 13000.0
            println("$(E_min) | $(folder)$(file)")
        end
    end
end

12993.722369664709 | ../../Data/hpc_out/rwm_ma_3_6r7m//11_rwm_ma_3_6r7m.jld2
12998.69379283667 | ../../Data/hpc_out/rwm_ma_3_6r7m//20_rwm_ma_3_6r7m.jld2
12985.092371807956 | ../../Data/hpc_out/rwm_ma_3_6r7m//34_rwm_ma_3_6r7m.jld2
12984.928372519898 | ../../Data/hpc_out/rwm_ma_3_6r7m//48_rwm_ma_3_6r7m.jld2
12999.411312928578 | ../../Data/hpc_out/rwm_ma_3_6r7m//50_rwm_ma_3_6r7m.jld2
12991.82647684909 | ../../Data/hpc_out/rwm_ma_3_6r7m//57_rwm_ma_3_6r7m.jld2
12998.101993074562 | ../../Data/hpc_out/rwm_ma_3_6r7m//69_rwm_ma_3_6r7m.jld2
12978.52185657988 | ../../Data/hpc_out/rwm_ma_3_6r7m//87_rwm_ma_3_6r7m.jld2


In [ ]:
#file = "../SolSim.jl_dev/io/hpc/all_data/three_monomers/protor_oj_0_0_os_0_85/6r7m/5/75_6r7m_protor_oj_0_0_os_0_85.jld2"
file = "../../Data/hpc_out/rwm_ma_3_6r7m/11_rwm_ma_3_6r7m.jld2"
@load file input output

mindex = argmin(output["Es"])
simulated_assembly_state = output["states"][mindex]

In [19]:
R1 = RotMatrix(@SMatrix[1.000000  0.000000  0.000000; 0.000000  1.000000  0.000000; 0.000000  0.000000  1.000000])
T1 = @SVector[0.00000, 0.00000, 0.00000]

R2 = RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000])
T2 = @SVector[-69.28043, 195.91352, -49.35000]

R3 = RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000])
T3 = @SVector[-61.30589, 104.11829, -47.94000]

R4 = RotMatrix(@SMatrix[0.992567  0.121696  0.000000; -0.121696  0.992567  0.000000; 0.000000  0.000000  1.000000])
T4 = @SVector[-19.48193, 22.01644, -46.53000]

3-element SVector{3, Float64} with indices SOneTo(3):
 -19.48193
  22.01644
 -46.53

In [42]:
template_centers, _ = SolSim.Utilities.poly_to_vec("polys/experimental/6r7m_protor.poly") 
molecule_experimental = deepcopy(molecules)
molecule_experimental.template_centers = template_centers
molecule_experimental.translations = [T2, T3, T4]
molecule_experimental.rotations = [R2, R3, R4]

radii = [[molecule_experimental.radii; molecule_experimental.radii]; molecule_experimental.radii]

SolSim.Utilities.vec_to_poly(SolSim.Utilities.realize_molecules(molecule_experimental), radii, "x1")
SolSim.Utilities.vec_to_poly(SolSim.Utilities.realize_molecules(molecules), radii, "x2")

In [6]:
function get_matched_distances_between_transformation_offsets(mol_a::SolSim.DataStructures.Molecules, mol_b::SolSim.DataStructures.Molecules, subset_a::Vector{Int}, subset_b::Vector{Int})
    function transform_dist(mol, idx, ss)
        euclidean(mol.translations[ss[1]] + mol.rotations[ss[1]] * mol.template_centers[idx], mol.translations[ss[2]] + mol.rotations[ss[2]] * mol.template_centers[idx])
    end

    @assert length(mol_a.template_centers) == length(mol_b.template_centers) 
    @assert length(mol_a.translations) == length(mol_b.translations)
    [abs(transform_dist(mol_a, i, subset_a) - transform_dist(mol_b, i, subset_b)) for i in 1:length(mol_a.template_centers)]
end

get_matched_distances_between_transformation_offsets (generic function with 1 method)

In [7]:
function sum_of_permutation(mol_a::SolSim.DataStructures.Molecules, mol_b::SolSim.DataStructures.Molecules, perm_a::Vector{Int}, perm_b::Vector{Int})
    n = length(mol_a.radii)
    d1 = sum(get_matched_distances_between_transformation_offsets(mol_a, mol_b, perm_a[[1,2]], perm_b[[1,2]])) / n
    d2 = sum(get_matched_distances_between_transformation_offsets(mol_a, mol_b, perm_a[[2,3]], perm_b[[2,3]])) / n
    d3 = sum(get_matched_distances_between_transformation_offsets(mol_a, mol_b, perm_a[[1,3]], perm_b[[1,3]])) / n
    println("$(d1), $(d2), $(d3)")
    d1 + d2 + d3
end

sum_of_permutation (generic function with 1 method)

In [48]:
permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3]]
for perm in permutations
    println(sum_of_permutation(molecule_experimental, molecules, [1, 2, 3], perm))
end

1.3357315185670564, 18.644756062991142, 22.577734537077223
42.55822211863542
1.2132546863901463, 18.644756062991142, 22.705511868858935
42.56352261824023
1.3357315185670564, 1.2134092426565837, 2.724863466675164
5.2740042278988035


In [5]:
R1 = RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000])
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000])
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = RotMatrix(@SMatrix[0.803441  0.595384  0.000000; -0.595384  0.803441  0.000000; 0.000000  0.000000  1.000000])
T3 = @SVector[-67.99970, 135.02619, -25.38000]

3-element SVector{3, Float64} with indices SOneTo(3):
 -67.9997
 135.02619
 -25.38

In [9]:
template_centers, _ = SolSim.Utilities.poly_to_vec("polys/experimental/6r7m_protor.poly") 
molecule_experimental = deepcopy(molecules)
molecule_experimental.template_centers = template_centers
molecule_experimental.translations = [T1, T2, T3]
molecule_experimental.rotations = [R1, R2, R3]

radii = [[molecule_experimental.radii; molecule_experimental.radii]; molecule_experimental.radii]

SolSim.Utilities.vec_to_poly(SolSim.Utilities.realize_molecules(molecule_experimental), radii, "x1")
SolSim.Utilities.vec_to_poly(SolSim.Utilities.realize_molecules(molecules), radii, "x2")

In [11]:
permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
for perm in permutations
    println(sum_of_permutation(molecule_experimental, molecules, [1, 2, 3], perm))
end

1.0344973812636278, 1.7526116493855854, 2.8837612501625696
5.670870280811783
4.658616153666888, 1.7526116493855854, 7.178012718919318
13.589240521971792
1.0344973812636278, 1.5562177641620782, 6.000170990245429
8.590886135671136
5.506623735765095, 1.5562177641620782, 7.178012718919318
14.240854218846492
4.658616153666888, 5.077931635208456, 6.000170990245429
15.736718779120775
5.506623735765095, 5.077931635208456, 2.8837612501625696
13.46831662113612
